In [11]:
# الخلية 1: استيراد المكتبات
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

In [12]:
# الخلية 2: تحميل البيانات النظيفة
data = pd.read_csv('../data/cleaned_language_data.csv')
data['cleaned_text'] = data['cleaned_text'].fillna('')  # معالجة القيم المفقودة في النصوص
print("📊 البيانات النظيفة:", data.shape)


📊 البيانات النظيفة: (10271, 3)


In [13]:
# الخلية 3: تقسيم البيانات
X = data['cleaned_text']
y = data['Language']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"📚 بيانات التدريب: {X_train.shape[0]}")
print(f"🧪 بيانات الاختبار: {X_test.shape[0]}")


📚 بيانات التدريب: 8216
🧪 بيانات الاختبار: 2055


In [14]:
# الخلية 4: تحويل النصوص إلى متجهات رقمية
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=5000,
    stop_words=None
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print(f"🔢 شكل بيانات التدريب بعد التحويل: {X_train_vec.shape}")
print(f"🔢 شكل بيانات الاختبار بعد التحويل: {X_test_vec.shape}")

🔢 شكل بيانات التدريب بعد التحويل: (8216, 5000)
🔢 شكل بيانات الاختبار بعد التحويل: (2055, 5000)


In [15]:
# الخلية 5: تجربة نماذج مختلفة
models = {
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

results = {}

for name, model in models.items():
    print(f"🔁 تدريب {name}...")
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    print(f"📊 دقة {name}: {accuracy:.4f}")

🔁 تدريب Naive Bayes...
📊 دقة Naive Bayes: 0.9401
🔁 تدريب Logistic Regression...
📊 دقة Logistic Regression: 0.9397
🔁 تدريب Random Forest...
📊 دقة Random Forest: 0.9202


In [16]:
# الخلية 6: اختيار أفضل نموذج
best_model_name = max(results, key=results.get)
best_model = models[best_model_name]

print(f"\n🎯 أفضل نموذج: {best_model_name} بدقة {results[best_model_name]:.4f}")


🎯 أفضل نموذج: Naive Bayes بدقة 0.9401


In [17]:
# الخلية 7: حفظ النموذج وأداة التحويل
joblib.dump(best_model, '../models/language_model.pkl')
joblib.dump(vectorizer, '../models/vectorizer.pkl')

print("💾 تم حفظ النموذج وأداة التحويل")


💾 تم حفظ النموذج وأداة التحويل


In [18]:
# الخلية 8: تقرير التدريب
training_report = {
    "best_model": best_model_name,
    "best_accuracy": results[best_model_name],
    "all_models_accuracy": results,
    "training_samples": X_train_vec.shape[0],
    "test_samples": X_test_vec.shape[0],
    "features_count": X_train_vec.shape[1]
}

print("\n📋 تقرير التدريب:")
for key, value in training_report.items():
    print(f"{key}: {value}")

print("\n✅ تم الانتهاء من تدريب النموذج!")


📋 تقرير التدريب:
best_model: Naive Bayes
best_accuracy: 0.9401459854014599
all_models_accuracy: {'Naive Bayes': 0.9401459854014599, 'Logistic Regression': 0.9396593673965937, 'Random Forest': 0.9201946472019464}
training_samples: 8216
test_samples: 2055
features_count: 5000

✅ تم الانتهاء من تدريب النموذج!
